In [1]:
# If installing from clean image
#!pip install dimcli -U
# !pip install dimcli plotly tqdm -U --quiet
#!pip install openpyxl -U
#!pip install dimcli plotly tqdm -U 


In [7]:
import numpy as np
import sympy as sp
import pickle
from IPython.display import HTML
import pandas as pd
import itertools
import dimcli
import openpyxl
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Credentials
dimcli.login(key="F86D0EE87BF7475E95D9A47574175BC3",
             endpoint="https://app.dimensions.ai")

Dimcli - Dimensions API Client (v0.9.1)
Connected to: https://app.dimensions.ai - DSL v1.31
Method: manual login


In [8]:
dsl = dimcli.Dsl()

In [9]:
GRIDID = "grid.427695.b"

In [10]:
tot = dsl.query(f"""search publications where research_orgs.id="{GRIDID}" return publications limit 1""", verbose=False).count_total
print(f"{GRIDID} has a total of {tot} publications in Dimensions")

grid.427695.b has a total of 292 publications in Dimensions


In [12]:
df = dsl.query(f"""search publications where research_orgs.id="{GRIDID}" return year limit 100""").as_dataframe()
df.rename(columns={"id": "year"}, inplace=True)
df.head(1)

Returned Year: 18
Time: 1.13s


,year,count
0,2014,28


In [17]:
data = dsl.query(f"""search publications
                where research_orgs.id="{GRIDID}"
                return publications[doi+title+times_cited+category_for+journal]
                sort by times_cited limit 1000""")
df = data.as_dataframe()
df.head(1)[['title', 'doi', 'times_cited', 'journal.title']]


Returned Publications: 292 (total = 292)
Time: 1.42s


,title,doi,times_cited,journal.title
0,Global surveillance of cancer survival 1995–20...,10.1016/s0140-6736(14)62038-9,1408,The Lancet


In [27]:
fundersdata = dsl.query(f"""search grants
                        where research_orgs.id="{GRIDID}"
                        return funders aggregate funding
                        sort by funding""")
df = fundersdata.as_dataframe()
df.head(10)

Returned Funders: 2
Time: 1.16s


,id,count,funding,types,latitude,city_name,state_name,country_name,name,linkout,longitude,acronym
0,grid.453129.8,2,1011852.0,[Government],-33.883300,Sydney,New South Wales,Australia,Cancer Australia,[http://canceraustralia.gov.au/],151.20883,CA
1,grid.413452.5,1,155950.0,[Government],-35.282425,Canberra,Australian Capital Territory,Australia,Australian Research Council,[http://www.arc.gov.au/],149.12405,ARC


In [42]:
authors = pd.read_excel("./ci-researcher-data/Premiers_Fellowship_Dimensions.xlsx")

In [43]:
researcherIDs = authors.researcher_ids

In [44]:
t = researcherIDs

In [47]:
authors.head(1)

,researcher_ids,Program,ProjectIdentifier,Title,ContactFullName,PrimaryOrganisationName,2020_TotalPubs,LTD_TotalPubs,2020_TotalCitations,LTD_TotalCitations,h-index
0,['ur.0731267322.21'],Career Development Fellowship - CDF,2018/CDF001,The Clincial Translation of NSW Invented Tumou...,Ricky O'Brien,The University of Sydney,NaN,NaN,NaN,NaN,NaN


In [59]:
authors.researcher_ids = authors.researcher_ids.str.replace(r"\['", "")
authors.researcher_ids = authors.researcher_ids.str.replace(r"\']", "")
authors.researcher_ids = authors.researcher_ids.str.replace(r"\'", " ")

In [76]:
authorIDs = authors.researcher_ids.to_list()
authorIDs = [str(x) for x in authorIDs]
[x for x in authorIDs if x != 'nan']

['ur.0731267322.21',
 'ur.01351637340.27   ur.015600604603.79',
 'ur.0714460423.40',
 'ur.01032446376.49',
 'ur.01367250265.00',
 'ur.0626251355.67',
 'ur.01332477657.37',
 'ur.01106242365.13',
 'ur.0764753454.22',
 'ur.0610762237.02',
 'ur.01210752347.03',
 'ur.0725316625.25',
 'ur.01265216223.88',
 'ur.01254371202.88',
 'ur.01310163707.98',
 'ur.01260235463.29',
 'ur.01107253763.12',
 'ur.01077011303.24',
 'ur.0706310077.68',
 'ur.01250725106.05',
 'ur.0713321776.37   ur.0650422353.85   ur.01121702557.52 \n  ur.07562506163.15',
 'ur.0733110542.08',
 'ur.0621053147.56',
 'ur.01154317624.53',
 'ur.01136576210.58',
 'ur.010222236545.28',
 'ur.01101265445.84   ur.01224157672.54',
 'ur.01077436063.62',
 'ur.01103675204.68',
 'ur.0740221370.50',
 'ur.01277214241.63',
 'ur.013451323244.74',
 'ur.01175620171.15',
 'ur.01367245642.54',
 'ur.016514611447.20',
 'ur.01365124445.22',
 'ur.0643042060.33',
 'ur.0725424627.42',
 'ur.01114472142.31',
 'ur.0656253324.38',
 'ur.014003773667.34',
 'ur.0

In [89]:
%%dsldf

search publications
where researchers.id = "ur.01204731133.77"
return publications[doi + researchers]
limit 1

Returned Publications: 1 (total = 33)
Time: 1.09s


,researchers,doi
0,"[{'id': 'ur.0715032760.11', 'research_orgs': [...",10.1186/s13073-021-00951-6


In [92]:
data = dsl.query("""search publications
                where researchers.id = "ur.01204731133.77"
                return publications[doi+title+times_cited+category_for+journal + authors]
                sort by times_cited limit 1000""")

"""hello""" + "by"

df = data.as_dataframe()
df


Returned Publications: 33 (total = 33)
Time: 1.13s


,times_cited,title,category_for,doi,authors,journal.id,journal.title
0,634,Global Epidemiology of Campylobacter Infection,"[{'id': '3177', 'name': '1117 Public Health an...",10.1128/cmr.00006-15,[{'raw_affiliation': ['School of Biotechnology...,jour.1098275,Clinical Microbiology Reviews
1,209,Faecal Microbiota Transplantation for Inflamma...,"[{'id': '3177', 'name': '1117 Public Health an...",10.1093/ecco-jcc/jjx063,[{'raw_affiliation': ['Inflammatory Bowel Dise...,jour.1039135,Journal of Crohn's and Colitis
2,114,Dual role of Helicobacter and Campylobacter sp...,"[{'id': '3114', 'name': '1108 Medical Microbio...",10.1136/gutjnl-2015-310545,[{'raw_affiliation': ['School of Biotechnology...,jour.1077125,Gut
3,87,Pattern-Recognition Receptors and Gastric Cancer,"[{'id': '2211', 'name': '11 Medical and Health...",10.3389/fimmu.2014.00336,[{'raw_affiliation': ['School of Biotechnology...,jour.1045166,Frontiers in Immunology
4,81,Dysbiosis of the microbiome in gastric carcino...,"[{'id': '2634', 'name': '0605 Microbiology'}, ...",10.1038/s41598-017-16289-2,[{'raw_affiliation': ['School of Biotechnology...,jour.1045337,Scientific Reports
5,72,TIRAP (MAL) S180L polymorphism is a common pro...,"[{'id': '2620', 'name': '0604 Genetics'}, {'id...",10.1016/j.meegid.2008.03.001,[{'raw_affiliation': ['Cellular Biology and Im...,jour.1027256,Infection Genetics and Evolution
6,63,"The Role of TLR2, TLR4 and CD14 Genetic Polymo...","[{'id': '3177', 'name': '1117 Public Health an...",10.1371/journal.pone.0060327,[{'raw_affiliation': ['School of Biotechnology...,jour.1037553,PLOS ONE
7,59,The NOD-Like Receptor Signalling Pathway in He...,"[{'id': '2620', 'name': '0604 Genetics'}, {'id...",10.1371/journal.pone.0098899,[{'raw_affiliation': ['School of Biotechnology...,jour.1037553,PLOS ONE
8,56,Meta-analysis of HLA-DRB1 and HLA-DQB1 polymor...,"[{'id': '2211', 'name': '11 Medical and Health...",10.1016/j.autrev.2007.12.002,[{'raw_affiliation': ['Cellular Biology and Im...,jour.1029983,Autoimmunity Reviews
9,49,Signatures within the esophageal microbiome ar...,"[{'id': '2634', 'name': '0605 Microbiology'}, ...",10.1186/s40168-018-0611-4,[{'raw_affiliation': ['Systems Biology Initiat...,jour.1048878,Microbiome
